# Get the ticker names for all listed companies

https://www.nasdaq.com/market-activity/stocks/screener?exchange=NASDAQ&render=download

In [1]:
import pandas as pd

In [2]:
df_nasdaq = pd.read_csv('nasdaq_screener_1665824524262.csv')
df_nasdaq['Listing'] = ['NASDAQ']*len(df_nasdaq)
df_nyse = pd.read_csv('nasdaq_screener_1665824563865.csv')
df_nyse['Listing'] = ['NYSE']*len(df_nyse)
df_amex = pd.read_csv('nasdaq_screener_1665824575784.csv')
df_amex['Listing'] = ['AMEX']*len(df_amex)

In [3]:
df_companies = pd.concat([df_nasdaq,df_nyse,df_amex],ignore_index=True)

In [4]:
len(df_companies)

8276

# Get Ticker CIK Map

https://www.sec.gov/include/ticker.txt

In [5]:
df_cik = pd.read_csv('TICKER_CIK_MAP.csv')
df_companies.dropna(subset=['Symbol'],inplace=True)
df_companies['Symbol']=df_companies['Symbol'].astype("string")
df_cik['Ticker']=df_cik['Ticker'].astype('string')

In [6]:
def getcik(symbol):
    #print(symbol)
    symbol = symbol.lower()
    if symbol in df_cik['Ticker'].values:
        cik = str(df_cik[df_cik['Ticker'] == symbol]['CIK'].values[0])
        cik = '0'*(10-len(cik))+cik
        return cik
    else:
        return 'EMPTY'

In [7]:
df_companies['CIK'] = df_companies['Symbol'].apply(lambda x:getcik(x))

Get Ticker Map from API

In [9]:
import requests
headers = {'User-Agent': "venkatesh.surampally79@gmail.com"}
tickers_cik = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers)
tickers_cik =     pd.json_normalize(pd.json_normalize(tickers_cik.json(),\
max_level=0).values[0])
tickers_cik["cik_str"] = tickers_cik["cik_str"].astype(str).str.zfill(10)
tickers_cik.set_index("ticker",inplace=True)

In [10]:
tickers_cik

,cik_str,title
ticker,,
AAPL,0000320193,Apple Inc.
MSFT,0000789019,MICROSOFT CORP
AMZN,0001018724,AMAZON COM INC
BRK-B,0001067983,BERKSHIRE HATHAWAY INC
UNH,0000731766,UNITEDHEALTH GROUP INC
...,...,...
SPIR-WT,0001816017,"Spire Global, Inc."
GHACU,0001806156,Gaming & Hospitality Acquisition Corp.
GLS-WT,0001805087,"GELESIS HOLDINGS, INC."


Companies Bio Data

In [11]:
df_companies.to_csv('companies_biodata.csv',index=False)

In [12]:
df_companies = pd.read_csv('companies_biodata.csv')

In [13]:
df_companies[df_companies['CIK'] == '0000320193']

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry,Listing,CIK
7,AAPL,Apple Inc. Common Stock,$138.38,-4.61,-3.224%,2.399141e+12,United States,1980.0,88508322,Technology,Computer Manufacturing,NASDAQ,0000320193


# Get Filings From sec.gov

https://www.sec.gov/Archives/edgar/daily-index/xbrl/companyfacts.zip

In [14]:
def get_info(data):
    df_final = pd.DataFrame()
    import datetime
    #Values that can be distributed across the quarters independent of each quarter
    import datetime
    #Values that can be distributed across the quarters independent of each quarter
    start = None
    end = None
    import datetime
    for lookup in ['OperatingExpenses','ResearchAndDevelopmentExpense','SellingGeneralAndAdministrativeExpense','NetIncomeLoss','Assets','AssetsCurrent','AssetsNoncurrent']:
        if lookup in data['facts']['us-gaap'].keys():
            assets_timeserie = pd.json_normalize(data['facts']['us-gaap'][lookup]["units"]["USD"])
            assets_timeserie["filed"] = pd.to_datetime(assets_timeserie["filed"])
            assets_timeserie["end"] = pd.to_datetime(assets_timeserie["end"])
            try:
                assets_timeserie["start"] = pd.to_datetime(assets_timeserie["start"])
                if start: start = min(start,assets_timeserie['start'].min())
                else: start = assets_timeserie['start'].min()
            except:
                if start: start = min(start,assets_timeserie['end'].min())
                else: start = assets_timeserie['end'].min()
            if end: end = max(end,assets_timeserie['end'].max())
            else: end = assets_timeserie['end'].max()
    df_final.index = pd.date_range(start.date()+ datetime.timedelta(days=7),end.date() + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist()
    
    for lookup in ['OperatingExpenses','ResearchAndDevelopmentExpense','SellingGeneralAndAdministrativeExpense','NetIncomeLoss']:
        if lookup in data['facts']['us-gaap'].keys():
            assets_timeserie = pd.json_normalize(data['facts']['us-gaap'][lookup]["units"]["USD"])
            assets_timeserie["filed"] = pd.to_datetime(assets_timeserie["filed"])
            assets_timeserie["start"] = pd.to_datetime(assets_timeserie["start"])
            assets_timeserie["end"] = pd.to_datetime(assets_timeserie["end"])
            assets_timeserie = assets_timeserie.sort_values("end")

            df_10Q_10K=assets_timeserie[assets_timeserie['form'].isin(['10-Q','10-K'])].reset_index()
            df_10Q=assets_timeserie[assets_timeserie['form'].isin(['10-Q'])].reset_index()
            df_10K=assets_timeserie[assets_timeserie['form'].isin(['10-K'])].reset_index()

            filed_quartes_val = {x:None for x in (pd.date_range(df_10Q_10K['start'].min().date()+ datetime.timedelta(days=7),df_10Q_10K['end'].max().date() + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist())}

            import datetime
            max_len = 0
            for i in range(len(df_10Q_10K)):
                l = pd.date_range(df_10Q_10K.iloc[i].start + datetime.timedelta(days=7),df_10Q_10K.iloc[i].end + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist()
                max_len = max(max_len,len(l))
            done = set()
            for length in range(1,max_len+1):
                for i in range(len(df_10Q_10K)):
                    l = pd.date_range(df_10Q_10K.iloc[i].start + datetime.timedelta(days=7),df_10Q_10K.iloc[i].end + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist()
                    if len(l) == length:
                        left = set(l) - done
                        comp = list(set(l) - left)
                        compsum = 0
                        for quarter in comp:
                            compsum += filed_quartes_val[quarter]         
                        for quarter in left:
                            distribute = (int(df_10Q_10K.iloc[i].val)-compsum)/len(left)
                            done.add(quarter)
                            filed_quartes_val[quarter] = distribute 
            df_final[lookup] = pd.DataFrame.from_dict(filed_quartes_val,columns=[lookup], orient='index')[lookup]
        

    #Values that dependent of each quarter like assets where assets from previous quarter cis transferred to next quarter
    for lookup in ['Assets','AssetsCurrent','AssetsNoncurrent']:
        if lookup in data['facts']['us-gaap'].keys():
            assets_timeserie = pd.json_normalize(data['facts']['us-gaap'][lookup]["units"]["USD"])
            assets_timeserie["filed"] = pd.to_datetime(assets_timeserie["filed"])
            assets_timeserie["start"] = pd.to_datetime(assets_timeserie["end"])
            assets_timeserie["end"] = pd.to_datetime(assets_timeserie["end"])
            assets_timeserie = assets_timeserie.sort_values("end")

            df_10Q_10K=assets_timeserie[assets_timeserie['form'].isin(['10-Q','10-K'])].reset_index()
            df_10Q=assets_timeserie[assets_timeserie['form'].isin(['10-Q'])].reset_index()
            df_10K=assets_timeserie[assets_timeserie['form'].isin(['10-K'])].reset_index()

            filed_quartes_val = {x:None for x in (pd.date_range(df_10Q_10K['start'].min().date()- datetime.timedelta(days=7),df_10Q_10K['end'].max().date() + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist())}

            import datetime
            max_len = 0
            for i in range(len(df_10Q_10K)):
                l = pd.date_range(df_10Q_10K.iloc[i].start - datetime.timedelta(days=7),df_10Q_10K.iloc[i].end + datetime.timedelta(days=7) + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist()
                max_len = max(max_len,len(l))
            done = set()
            for length in range(1,max_len+1):
                for i in range(len(df_10Q_10K)):
                    l = pd.date_range(df_10Q_10K.iloc[i].start - datetime.timedelta(days=7),df_10Q_10K.iloc[i].end + datetime.timedelta(days=7) + pd.offsets.QuarterBegin(1), freq='Q').strftime('%B %Y').tolist()
                    if len(l) == length:
                        left = set(l) - done
                        comp = list(set(l) - left)     
                        for quarter in left:
                            distribute = (int(df_10Q_10K.iloc[i].val))/1
                            done.add(quarter)
                            filed_quartes_val[quarter] = distribute 
            df_final[lookup] = pd.DataFrame.from_dict(filed_quartes_val,columns=[lookup], orient='index')[lookup]
        
    return df_final
    

In [15]:
cd C:\Users\User\Desktop\Workflow Assignment

C:\Users\User\Desktop\Workflow Assignment


In [16]:
#Generate 10-K and 10-Q facts

import sys
import os
def print_progress_bar(index, total, label,time,speed):
    n_bar = 50  # Progress bar width
    progress = index / total
    sys.stdout.write('\r')
    sys.stdout.write(f"[{'=' * int(n_bar * progress):{n_bar}s}] {int(100 * progress)}%  {label}, VOID = {time}, {speed}")
    sys.stdout.flush()


import json
found = 0
not_found = 0
no_cik = 0
void = 0
voids = []
error = []
tot = len(os.listdir(path=r'C:\Users\User\Desktop\Workflow Assignment\XBLR_File')[:])
for j,cik in enumerate(os.listdir(path=r'C:\Users\User\Desktop\Workflow Assignment\XBLR_File')[:]):
    cik = cik[3:-5]
    if cik != 'EMPTY':
        try:
            f = open('XBLR_File\CIK'+str(cik)+'.json')
            data = json.load(f)
            f.close()
            found += 1
            df_out = get_info(data)
            df_out.to_csv('10K_RESULTS\CIK'+cik+'_VALUES.csv')
            if len(df_out)==0:
                void+=1
                voids.append(cik)
            if ((df_final.isnull().count() - df_final.isnull().sum())==0).sum() >0:
                error.append(cik)
        except:
            not_found += 1
    else:
        no_cik += 1
    print_progress_bar(j, tot-1, "Generating Testing Data, Error: "+str(len(error)),void, 'Done ' + str(j+1)+' Records Rem '+str(tot-1-j))

[==================================================] 100%  Generating Testing Data, Error: 0, VOID = 0, Done 16556 Records Rem 000

# We can also get the company data with API (Not Implemented)

In [17]:
tickers_cik = pd.read_csv('listed_companies.csv')
tickers_cik['cik_str'] = tickers_cik['cik_str'].astype('string')

In [18]:
response = requests.get("https://data.sec.gov/api/xbrl/companyconcept/CIK0000320193/us-gaap/Assets.json", headers=headers)

In [19]:
assets_timeserie = pd.json_normalize(response.json()["units"]["USD"])
assets_timeserie["filed"] = pd.to_datetime(assets_timeserie["filed"])
assets_timeserie = assets_timeserie.sort_values("end")

In [20]:
assets_timeserie

,end,val,accn,fy,fp,form,filed,frame
0,2008-09-27,39572000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN
1,2008-09-27,39572000000,0001193125-09-214859,2009,FY,10-K,2009-10-27,NaN
2,2008-09-27,36171000000,0001193125-10-012091,2009,FY,10-K/A,2010-01-25,NaN
3,2008-09-27,36171000000,0001193125-10-238044,2010,FY,10-K,2010-10-27,CY2008Q3I
4,2009-06-27,48140000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I
...,...,...,...,...,...,...,...,...
109,2021-09-25,351002000000,0000320193-22-000059,2022,Q2,10-Q,2022-04-29,NaN
110,2021-09-25,351002000000,0000320193-22-000070,2022,Q3,10-Q,2022-07-29,CY2021Q3I
111,2021-12-25,381191000000,0000320193-22-000007,2022,Q1,10-Q,2022-01-28,CY2021Q4I
112,2022-03-26,350662000000,0000320193-22-000059,2022,Q2,10-Q,2022-04-29,CY2022Q1I
